# 1. 라이브러리 선언하기

In [5]:
import pandas as pd
import numpy as np

# 2. 데이터 불러오기

In [6]:
selloutData = \
    pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

In [7]:
selloutTypeData = selloutData.astype({
    "REGIONID":str,
    "PRODUCT":str,
    "YEARWEEK":str,
    "QTY":float})

## 3-1. 불량데이터 처리

In [8]:
selloutTypeData["QTY_NEW"]= np.where(selloutTypeData.QTY<0,0,selloutTypeData.QTY)

In [9]:
selloutTypeData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW
0,A60,PRODUCT4,201402,71.0,71.0
1,A60,PRODUCT59,201402,22275.0,22275.0
2,A60,PRODUCT34,201402,4463.0,4463.0
3,A60,PRODUCT47,201402,0.0,0.0
4,A60,PRODUCT56,201402,23.0,23.0
...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,824.0
124654,A10,PRODUCT56,201630,275.0,275.0
124655,A10,PRODUCT61,201630,0.0,0.0
124656,A10,PRODUCT12,201630,15021.0,15021.0


## 3-2. 데이터 통합

In [10]:
## YEARWEEK에서 나눠야할 연도, 주차 길이 정의
yearAndWeek = 4

## YearAndWeek를 기준으로 데이터 나누기
selloutTypeData["YEAR"]=selloutTypeData.YEARWEEK.str[:yearAndWeek]
selloutTypeData["WEEK"]=selloutTypeData.YEARWEEK.str[yearAndWeek:]

## 구하고자 하는 Week 변수 지정
yearlength = 52

## yearlength를 기준으로 데이터 찾기
findSelloutData = selloutTypeData.loc[selloutTypeData.WEEK.astype(int)<=yearlength]

## 인덱스 재설정
refinedSelloutData=findSelloutData.reset_index(drop=True)

## 검산식
# refinedSelloutData.loc[refinedSelloutData.WEEK.astype(int)>yearlength]

refinedSelloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,02
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02
3,A60,PRODUCT47,201402,0.0,0.0,2014,02
4,A60,PRODUCT56,201402,23.0,23.0,2014,02
...,...,...,...,...,...,...,...
123859,A10,PRODUCT60,201630,824.0,824.0,2016,30
123860,A10,PRODUCT56,201630,275.0,275.0,2016,30
123861,A10,PRODUCT61,201630,0.0,0.0,2016,30
123862,A10,PRODUCT12,201630,15021.0,15021.0,2016,30


# 4. Group By 집계함수

<!-- ## 3. 불량 데이터 처리

selloutTypeData["QTY_NEW"] = np.where(selloutTypeData.QTY<0,0,selloutTypeData.QTY)
selloutTypeData -->

In [11]:
refinedSelloutData.columns

Index(['REGIONID', 'PRODUCT', 'YEARWEEK', 'QTY', 'QTY_NEW', 'YEAR', 'WEEK'], dtype='object')

## 4-1 지역, 상품, 연주차 컬럼순으로 오름차순 정렬하기

In [12]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]

sortedData = refinedSelloutData.sort_values(by=sortKey,
                                     ascending=[True,True,True],
                                    inplace=False, ignore_index=True)

## 4-2 판매량(qty_new)의 평균 연산

In [13]:
groupKey = ["REGIONID","PRODUCT","YEAR"]

groupData = sortedData.\
    groupby(by=groupKey)["QTY_NEW"].agg(["mean"])

groupData.rename(columns={"mean":"QTY_MEAN"},
                inplace=True)

# 컬럼명 변경(mean => QTY_MEAN)
groupData.reset_index(inplace=True)
groupData

## 4-3 이동집계함수

In [172]:
## 값이 1개라도 있을때, 현재 값을 중심값으로 이동평균 산출
sortedData["MA5"]=sortedData.QTY.\
            rolling(window=5, 
                     min_periods=1,
                    center=True).mean() 

In [172]:
## 값이 없으면 계산 x, 현재 값을 중심값으로 이동평균 산출
sortedData["MA5_BASIC"]=sortedData.QTY.\
            rolling(window=5, 
                    center=True).mean()

In [172]:
## 값이 없으면 계산 x, 현재 값을 마지막 값으로 상위 4개와 이동평균 산출
sortedData["MA5_BASIC_NO"]=sortedData.QTY.\
            rolling(window=5, 
                    center=False).mean()

In [149]:
sortedData["EFFECT"]= sortedData.QTY / sortedData.MA5

In [153]:
sortedData.drop(columns=["MA5_BASIC_NO"],inplace=True)

In [156]:
groupKey = ["REGIONID","PRODUCT","YEAR"]

In [161]:
## 3갸 연도의 지역, 주차, 상품, 주차별 평균 Effect를 구하고 싶다.
ratioData = sortedData.groupby(groupKey)[["EFFECT"]].agg("mean").reset_index()

In [163]:
ratioData.to_csv("d:/finalresult.csv",index=False, encoding="ms949")

In [176]:
sortedData.to_csv("d:/middleresult.csv",index=False, encoding="ms949")